In [ ]:
# RunPod A100 GPU - Könyvtárak telepítése és importálása
!pip install -U torch sentence-transformers accelerate pyarrow pandas tqdm transformers

import pandas as pd
import numpy as np
import gc
import json
import pyarrow as pa
import pyarrow.parquet as pq
from sentence_transformers import SentenceTransformer
import torch
import psutil
import time
import logging
from tqdm import tqdm
from typing import List, Dict, Any
import os
import warnings
warnings.filterwarnings('ignore')

# A100 GPU optimalizáció
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

print("RunPod A100 környezet inicializálva!")
print(f"CUDA elérhető: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memória: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f}GB")

In [ ]:
# RunPod A100 konfiguráció
print("RunPod A100 konfiguráció beállítása...")

# Fájl elérési utak RunPod-on
INPUT_CSV = "/workspace/cleaned_data_for_embedding.csv"
OUTPUT_PARQUET = "/workspace/processed_documents_with_embeddings.parquet"

# Qwen3-8B A100 optimalizált paraméterek
MODEL_NAME = "Qwen/Qwen3-Embedding-8B"
EMBEDDING_DIMENSION = 8192
BATCH_SIZE = 32            # A100 optimális
CHUNK_SIZE = 5000          # Chunk méret
MAX_TOKEN_LENGTH = 8192
USE_MIXED_PRECISION = True
MEMORY_LIMIT_GB = 75       # A100 80GB-ból 75GB használata

print(f"Bemeneti CSV: {INPUT_CSV}")
print(f"Kimeneti Parquet: {OUTPUT_PARQUET}")
print(f"Modell: {MODEL_NAME}")
print(f"Dimenzió: {EMBEDDING_DIMENSION}")
print(f"Batch méret: {BATCH_SIZE}")
print(f"Chunk méret: {CHUNK_SIZE:,}")
print(f"Mixed Precision: {USE_MIXED_PRECISION}")
print(f"Memória limit: {MEMORY_LIMIT_GB}GB")

# Logging konfiguráció
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('/workspace/embedding_generation.log')
    ]
)
logger = logging.getLogger(__name__)

In [ ]:
# CSV adatok betöltése és validálása
logger.info("CSV adatok validálása...")

# Fájl létezés ellenőrzése
if not os.path.exists(INPUT_CSV):
    raise FileNotFoundError(f"CSV fájl nem található: {INPUT_CSV}")

# Mintaadatok betöltése struktúra ellenőrzéshez
df_sample = pd.read_csv(INPUT_CSV, nrows=1000)
logger.info(f"Minta betöltve: {len(df_sample)} sor")

# Teljes fájl méret becslése
total_rows = sum(1 for _ in open(INPUT_CSV, 'r', encoding='utf-8')) - 1
logger.info(f"Becsült teljes sorok: {total_rows:,}")

# Kötelező oszlopok ellenőrzése
required_columns = ['text', 'doc_id']
missing_columns = [col for col in required_columns if col not in df_sample.columns]
if missing_columns:
    raise ValueError(f"Hiányzó kötelező oszlopok: {missing_columns}")

# Teljes metadata oszlop lista a preprocess_documents.py alapján
expected_metadata_columns = [
    'doc_id', 'text', 'birosag', 'JogTerulet', 'Azonosito', 'MeghozoBirosag',
    'EgyediAzonosito', 'HatarozatEve', 'AllKapcsolodoUgyszam', 'AllKapcsolodoBirosag',
    'KapcsolodoHatarozatok', 'Jogszabalyhelyek'
]

# Jelenlegi oszlopok listázása
available_columns = list(df_sample.columns)
metadata_columns_present = [col for col in expected_metadata_columns if col in available_columns]
metadata_columns_missing = [col for col in expected_metadata_columns if col not in available_columns]

print("CSV validáció sikeres!")
print(f"Teljes sorok: {total_rows:,}")
print(f"Összes oszlop: {len(available_columns)}")
print(f"Jelenlevő metadata oszlopok ({len(metadata_columns_present)}): {metadata_columns_present}")
if metadata_columns_missing:
    print(f"Hiányzó metadata oszlopok ({len(metadata_columns_missing)}): {metadata_columns_missing}")

# Szöveg hossz statisztikák
text_lengths = df_sample['text'].str.len()
print(f"\nSzöveg hossz statisztikák (minta):")
print(f"  Átlag: {text_lengths.mean():.0f} karakter")
print(f"  Medián: {text_lengths.median():.0f} karakter")
print(f"  Min: {text_lengths.min():.0f} karakter")
print(f"  Max: {text_lengths.max():.0f} karakter")

# Becsült feldolgozási idő
estimated_batches = (total_rows + BATCH_SIZE - 1) // BATCH_SIZE
estimated_chunks = (total_rows + CHUNK_SIZE - 1) // CHUNK_SIZE
print(f"\nBecsült feldolgozás:")
print(f"  Chunk-ok száma: {estimated_chunks:,}")
print(f"  Batch-ek száma: {estimated_batches:,}")
print(f"  Becsült idő: 2-3 óra A100 GPU-n")

In [ ]:
# Qwen3-Embedding-8B modell osztály A100-ra optimalizálva
logger.info("Qwen3-Embedding-8B modell osztály létrehozása...")

class OptimizedQwen3EmbeddingGenerator:
    def __init__(self):
        self.model_name = MODEL_NAME
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.dimension = EMBEDDING_DIMENSION
        self.max_tokens = MAX_TOKEN_LENGTH
        self.batch_size = BATCH_SIZE
        
        # Teljesítmény követés
        self.processed_count = 0
        self.failed_count = 0
        self.batch_times = []
        self.peak_memory_usage = 0
        
        logger.info(f"Device: {self.device}")
        
        try:
            # Modell betöltése
            logger.info("Qwen3-8B modell betöltése...")
            self.model = SentenceTransformer(
                self.model_name,
                device=self.device,
                trust_remote_code=True
            )
            
            # Mixed precision optimalizáció
            if self.device == 'cuda' and USE_MIXED_PRECISION:
                self.model.half()
                logger.info("Mixed precision bekapcsolva")
            
            # Modell warmup
            self._warmup_model()
            logger.info("Modell sikeresen betöltve és optimalizálva")
            
        except Exception as e:
            logger.error(f"Modell betöltési hiba: {e}")
            raise
    
    def _warmup_model(self):
        """Modell warmup konzisztens teljesítményért"""
        logger.info("Modell warmup...")
        dummy_texts = ["Ez egy teszt szöveg a modell bemelegítéséhez."] * 4
        
        with torch.cuda.amp.autocast(enabled=USE_MIXED_PRECISION):
            _ = self.model.encode(dummy_texts, show_progress_bar=False)
        
        self._cleanup_memory()
        logger.info("Warmup befejezve")
    
    def _cleanup_memory(self):
        """Memória tisztítás"""
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
    
    def _monitor_memory(self):
        """GPU memória monitoring"""
        if not torch.cuda.is_available():
            return {}
        
        allocated = torch.cuda.memory_allocated() / (1024**3)
        reserved = torch.cuda.memory_reserved() / (1024**3)
        
        self.peak_memory_usage = max(self.peak_memory_usage, allocated)
        
        return {
            'allocated_gb': allocated,
            'reserved_gb': reserved,
            'peak_usage_gb': self.peak_memory_usage
        }

# Embedding generátor inicializálása
embedding_generator = OptimizedQwen3EmbeddingGenerator()
print("Qwen3-8B modell sikeresen inicializálva!")
print(f"Dimenzió: {embedding_generator.dimension}")
print(f"Device: {embedding_generator.device}")
print(f"Mixed Precision: {USE_MIXED_PRECISION}")

In [ ]:
# Embedding generálás metódus hozzáadása
def generate_embeddings_batch(self, texts):
    """A100 optimalizált batch embedding generálás"""
    batch_start_time = time.time()
    
    try:
        # Szöveg előfeldolgozás
        processed_texts = []
        for text in texts:
            if len(text) > self.max_tokens * 3:  # ~3 char/token becslés
                text = text[:self.max_tokens * 3]
            processed_texts.append(text)
        
        # Mixed precision embedding generálás
        with torch.cuda.amp.autocast(enabled=USE_MIXED_PRECISION):
            embeddings = self.model.encode(
                processed_texts,
                normalize_embeddings=True,
                show_progress_bar=False,
                convert_to_numpy=True,
                batch_size=len(processed_texts)
            )
        
        # Dimenzió ellenőrzés és korrekció
        if embeddings.shape[1] != self.dimension:
            if embeddings.shape[1] > self.dimension:
                embeddings = embeddings[:, :self.dimension]
            else:
                padding = np.zeros((embeddings.shape[0], self.dimension - embeddings.shape[1]))
                embeddings = np.hstack([embeddings, padding])
        
        # Teljesítmény követés
        batch_time = time.time() - batch_start_time
        self.batch_times.append(batch_time)
        self.processed_count += len(texts)
        
        return embeddings.astype(np.float32)
        
    except Exception as e:
        logger.error(f"Batch feldolgozási hiba: {e}")
        self.failed_count += len(texts)
        # Fallback: NaN vektorok
        return np.full((len(texts), self.dimension), np.nan, dtype=np.float32)
    
    finally:
        # Rendszeres memória cleanup
        if self.processed_count % 1000 == 0:
            self._cleanup_memory()

# Metódus hozzáadása az osztályhoz
OptimizedQwen3EmbeddingGenerator.generate_embeddings_batch = generate_embeddings_batch

print("Embedding generálás metódus hozzáadva!")

In [ ]:
# Segédfüggvények
def create_metadata_json(row):
    """Teljes metadata JSON készítése az összes elérhető oszloppal"""
    metadata = {
        'doc_id': str(row.get('doc_id', '')),
        'birosag': str(row.get('birosag', '')),
        'JogTerulet': str(row.get('JogTerulet', '')),
        'Azonosito': str(row.get('Azonosito', '')),
        'MeghozoBirosag': str(row.get('MeghozoBirosag', '')),
        'EgyediAzonosito': str(row.get('EgyediAzonosito', '')),
        'HatarozatEve': str(row.get('HatarozatEve', '')),
        'AllKapcsolodoUgyszam': str(row.get('AllKapcsolodoUgyszam', '')),
        'AllKapcsolodoBirosag': str(row.get('AllKapcsolodoBirosag', '')),
        'KapcsolodoHatarozatok': str(row.get('KapcsolodoHatarozatok', '')),
        'Jogszabalyhelyek': str(row.get('Jogszabalyhelyek', '')),
        'text_length': len(str(row.get('text', ''))),
        'processed_timestamp': time.time()
    }
    return json.dumps(metadata, ensure_ascii=False)

def adaptive_batch_size(text_lengths, base_batch_size=BATCH_SIZE):
    """Adaptív batch méret szöveg hossz alapján"""
    avg_length = np.mean(text_lengths)
    
    if avg_length > 6000:
        return max(8, base_batch_size // 4)
    elif avg_length > 4000:
        return max(16, base_batch_size // 2)
    elif avg_length > 2000:
        return base_batch_size
    else:
        return min(64, base_batch_size * 2)

def prepare_final_columns(chunk_df):
    """Végső oszlopok előkészítése - összes metadata megőrzése"""
    # Alapvető oszlopok (kötelező)
    final_columns = ['doc_id', 'text', 'embedding', 'metadata_json']
    
    # Összes metadata oszlop hozzáadása, ha létezik
    metadata_columns = [
        'birosag', 'JogTerulet', 'Azonosito', 'MeghozoBirosag',
        'EgyediAzonosito', 'HatarozatEve', 'AllKapcsolodoUgyszam', 
        'AllKapcsolodoBirosag', 'KapcsolodoHatarozatok', 'Jogszabalyhelyek'
    ]
    
    # Csak a létező oszlopokat adjuk hozzá
    for col in metadata_columns:
        if col in chunk_df.columns:
            final_columns.append(col)
    
    # Visszaadjuk a létező oszlopokat
    available_columns = [col for col in final_columns if col in chunk_df.columns]
    return available_columns

print("Segédfüggvények betöltve!")

In [ ]:
# A100 főfolyamat - Robosztus embedding generálás
def process_embeddings_a100():
    """
    A100 GPU-ra optimalizált robosztus embedding generálás
    """
    
    start_time = time.time()
    logger.info("A100 embedding feldolgozás kezdése...")
    
    total_rows = sum(1 for _ in open(INPUT_CSV, 'r', encoding='utf-8')) - 1
    processed_rows = 0
    all_results = []
    
    logger.info(f"Feldolgozandó sorok: {total_rows:,}")
    logger.info(f"Chunk méret: {CHUNK_SIZE:,}")
    logger.info(f"Batch méret: {BATCH_SIZE}")
    
    # Chunk-alapú feldolgozás
    chunk_count = 0
    total_chunks = (total_rows + CHUNK_SIZE - 1) // CHUNK_SIZE
    
    with tqdm(total=total_chunks, desc="Chunk feldolgozás", unit="chunk") as chunk_pbar:
        
        for chunk_df in pd.read_csv(INPUT_CSV, chunksize=CHUNK_SIZE, encoding='utf-8'):
            chunk_count += 1
            chunk_start_time = time.time()
            
            # Adatok tisztítása
            original_len = len(chunk_df)
            chunk_df = chunk_df.dropna(subset=['text', 'doc_id'])
            chunk_df['text'] = chunk_df['text'].astype(str)
            chunk_df = chunk_df[chunk_df['text'].str.len() > 10]  # Min szöveghossz
            
            if len(chunk_df) == 0:
                logger.warning(f"Chunk {chunk_count}: nincs érvényes adat")
                chunk_pbar.update(1)
                continue
            
            logger.info(f"Chunk {chunk_count}/{total_chunks}: {len(chunk_df):,} érvényes sor")
            
            # Szövegek és adaptív batch méret
            texts = chunk_df['text'].tolist()
            text_lengths = [len(text) for text in texts]
            dynamic_batch_size = adaptive_batch_size(text_lengths, BATCH_SIZE)
            
            # Batch-es embedding generálás
            all_embeddings = []
            total_batches_in_chunk = (len(texts) + dynamic_batch_size - 1) // dynamic_batch_size
            
            with tqdm(total=total_batches_in_chunk, desc=f"Chunk {chunk_count} batch-ek", 
                     unit="batch", leave=False) as batch_pbar:
                
                for batch_idx in range(0, len(texts), dynamic_batch_size):
                    batch_texts = texts[batch_idx:batch_idx + dynamic_batch_size]
                    
                    # Embedding generálás hibakezeléssel
                    try:
                        batch_embeddings = embedding_generator.generate_embeddings_batch(batch_texts)
                        all_embeddings.extend(batch_embeddings.tolist())
                        
                        # Memória monitoring
                        memory_info = embedding_generator._monitor_memory()
                        if memory_info.get('allocated_gb', 0) > MEMORY_LIMIT_GB * 0.9:
                            logger.warning(f"Magas memória: {memory_info.get('allocated_gb', 0):.1f}GB")
                            embedding_generator._cleanup_memory()
                        
                    except Exception as e:
                        logger.error(f"Batch hiba: {e}")
                        # Fallback NaN vektorok
                        nan_embeddings = np.full((len(batch_texts), EMBEDDING_DIMENSION), np.nan)
                        all_embeddings.extend(nan_embeddings.tolist())
                    
                    batch_pbar.update(1)
            
            # Embedding számossági ellenőrzés
            if len(all_embeddings) != len(chunk_df):
                logger.error(f"Embedding számossági hiba: {len(all_embeddings)} != {len(chunk_df)}")
                # Kiegészítés NaN-okkal
                while len(all_embeddings) < len(chunk_df):
                    all_embeddings.append(np.full(EMBEDDING_DIMENSION, np.nan).tolist())
            
            # Eredmények hozzáadása
            chunk_df['embedding'] = all_embeddings
            chunk_df['metadata_json'] = chunk_df.apply(create_metadata_json, axis=1)
            
            # Végső oszlopok - összes metadata megőrzése
            available_columns = prepare_final_columns(chunk_df)
            chunk_result = chunk_df[available_columns].copy()
            
            all_results.append(chunk_result)
            processed_rows += len(chunk_df)
            
            # Progress update
            chunk_time = time.time() - chunk_start_time
            rows_per_sec = len(chunk_df) / chunk_time
            
            chunk_pbar.set_postfix({
                'Sorok/sec': f'{rows_per_sec:.1f}',
                'Memória': f'{embedding_generator._monitor_memory().get("allocated_gb", 0):.1f}GB',
                'Sikeres': embedding_generator.processed_count,
                'Hibás': embedding_generator.failed_count
            })
            chunk_pbar.update(1)
            
            # Rendszeres cleanup
            if chunk_count % 3 == 0:
                embedding_generator._cleanup_memory()
    
    # Eredmények egyesítése
    logger.info("DataFrame-ek egyesítése...")
    if not all_results:
        raise ValueError("Nincs feldolgozott adat!")
    
    final_df = pd.concat(all_results, ignore_index=True)
    logger.info(f"Egyesített DataFrame: {len(final_df):,} sor")
    
    return final_df, processed_rows, time.time() - start_time

# A100 főfolyamat indítása
logger.info("A100 embedding feldolgozás indítása...")
final_df, processed_rows, total_time = process_embeddings_a100()

In [ ]:
# Parquet mentés és végső validáció
logger.info("Parquet mentés és validáció...")

# Embedding validáció
valid_embeddings = 0
nan_embeddings = 0
dimension_errors = 0

for idx, emb in enumerate(final_df['embedding']):
    if isinstance(emb, list):
        if len(emb) == EMBEDDING_DIMENSION:
            if not np.any(np.isnan(emb)):
                valid_embeddings += 1
            else:
                nan_embeddings += 1
        else:
            dimension_errors += 1
    else:
        dimension_errors += 1

logger.info(f"Embedding validáció:")
logger.info(f"  Érvényes: {valid_embeddings:,}")
logger.info(f"  NaN: {nan_embeddings:,}")
logger.info(f"  Dimenzió hiba: {dimension_errors:,}")

# Parquet mentés
logger.info(f"Végső Parquet mentés: {OUTPUT_PARQUET}")

final_df.to_parquet(
    OUTPUT_PARQUET,
    engine='pyarrow',
    index=False,
    compression='snappy',
    row_group_size=50000
)

# Fájl validáció
file_size = os.path.getsize(OUTPUT_PARQUET) / (1024**3)

# Gyors visszaolvasási teszt
test_df = pd.read_parquet(OUTPUT_PARQUET, nrows=100)
logger.info(f"Visszaolvasási teszt sikeres: {len(test_df)} sor")

# Végső statisztikák
logger.info("A100 QWEN3-8B EMBEDDING GENERÁLÁS BEFEJEZVE!")
logger.info(f"Feldolgozott sorok: {processed_rows:,}")
logger.info(f"Végső sorok: {len(final_df):,}")
logger.info(f"Végső oszlopok ({len(final_df.columns)}): {list(final_df.columns)}")
logger.info(f"Érvényes embeddings: {valid_embeddings:,}")
logger.info(f"Fájl méret: {file_size:.2f}GB")
logger.info(f"Teljes futási idő: {total_time/3600:.2f} óra")

print("\n" + "="*80)
print("QWEN3-8B EMBEDDING FELDOLGOZÁS BEFEJEZVE!")
print("="*80)
print(f"📊 Feldolgozott dokumentumok: {processed_rows:,}")
print(f"📁 Végső Parquet fájl: {OUTPUT_PARQUET}")
print(f"📈 Oszlopok száma: {len(final_df.columns)}")
print(f"🎯 Érvényes embeddings: {valid_embeddings:,}")
print(f"💾 Fájl méret: {file_size:.2f}GB")
print(f"⏱️  Futási idő: {total_time/3600:.2f} óra")
print("="*80)
logger.info(f"Teljes futási idő: {total_time/3600:.2f} óra")
logger.info(f"Átlag sebesség: {processed_rows/total_time:.1f} sor/sec")
logger.info(f"Fájl méret: {file_size:.2f} GB")
logger.info(f"Csúcs memória: {embedding_generator.peak_memory_usage:.1f}GB")

print("\nA100 QWEN3-8B EMBEDDING GENERÁLÁS SIKERESEN BEFEJEZVE!")
print(f"Feldolgozott sorok: {processed_rows:,}")
print(f"Érvényes embeddings: {valid_embeddings:,}")
print(f"Fájl méret: {file_size:.2f} GB")
print(f"Teljes idő: {total_time/3600:.2f} óra")
print(f"Sebesség: {processed_rows/total_time:.1f} sor/sec")
print(f"Csúcs memória: {embedding_generator.peak_memory_usage:.1f}GB")
print(f"Sikerességi arány: {(valid_embeddings/len(final_df)*100):.1f}%")